### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M09D14', 'UTC 01:10')


# Settings

In [2]:
MAXTASKS = 200
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [8]:
detailedTasks

,creation_timestamp_ms,description,id,output_url,start_timestamp_ms,state,task_type,update_timestamp_ms,calctime(min),queuetime(min),runtime(min)
0,1505351319478,runoff_monthY2014M12V13,G6MEIOAE4RH4FF4XNY3TLXS5,https://console.developers.google.com/storage/...,1505351345519,COMPLETED,EXPORT_FEATURES,1505351347213,0.028233,0.434017,0.462250
1,1505351318567,runoff_monthY2014M11V13,IMOTSZIKJ7SKVHJOWJM4CATS,https://console.developers.google.com/storage/...,1505351339647,COMPLETED,EXPORT_FEATURES,1505351341685,0.033967,0.351333,0.385300
2,1505351317509,runoff_monthY2014M10V13,CPLPLF5HE2INWSKVXQB2IL3Q,https://console.developers.google.com/storage/...,1505351339238,COMPLETED,EXPORT_FEATURES,1505351341774,0.042267,0.362150,0.404417
3,1505351316551,runoff_monthY2014M09V13,Z23CMQ5AFNWGT5GRKGZCOSXY,https://console.developers.google.com/storage/...,1505351336185,COMPLETED,EXPORT_FEATURES,1505351337955,0.029500,0.327233,0.356733
4,1505351315525,runoff_monthY2014M08V13,PZK6VAWO2FY7FJBYHKJVP4FX,https://console.developers.google.com/storage/...,1505351333236,COMPLETED,EXPORT_FEATURES,1505351347649,0.240217,0.295183,0.535400
5,1505351314494,runoff_monthY2014M07V13,SIR6QCFVX3NHZQ66CU65FKE7,https://console.developers.google.com/storage/...,1505351330458,COMPLETED,EXPORT_FEATURES,1505351337689,0.120517,0.266067,0.386583
6,1505351313455,runoff_monthY2014M06V13,2WAYILMYMKI55XGHXKUCUVZ4,https://console.developers.google.com/storage/...,1505351330096,COMPLETED,EXPORT_FEATURES,1505351332491,0.039917,0.277350,0.317267
7,1505351312351,runoff_monthY2014M05V13,DDAMHGBA6NXIL4V4FVNQ3HXY,https://console.developers.google.com/storage/...,1505351327035,COMPLETED,EXPORT_FEATURES,1505351329570,0.042250,0.244733,0.286983
8,1505351311343,runoff_monthY2014M04V13,YZBHVUJFLQQ63GLKA7X5IOMO,https://console.developers.google.com/storage/...,1505351324118,COMPLETED,EXPORT_FEATURES,1505351326301,0.036383,0.212917,0.249300
9,1505351310420,runoff_monthY2014M03V13,ZOXMRYQOXNMVDFHF42TK4YLE,https://console.developers.google.com/storage/...,1505351321638,COMPLETED,EXPORT_FEATURES,1505351324093,0.040917,0.186967,0.227883


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)